In [6]:
# Import the dependencies.
import pandas as pd
import numpy as np
# Import the requests library.
import requests

# Import the API key.
import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))

# Now do your import
from config import weather_api_key
# from folder.config import weather_api_key

# looks for nearest city with geo coordinates
from citipy import citipy

In [8]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [9]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [10]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

770

In [11]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=3ef41da726500c6f691a33dccaeb7ff7


In [12]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [13]:
# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        #city_weather_condition = for item in city_weather["weather"]:print(item["description"])
        for item in city_weather["weather"]:
            city_weather_condition = item["description"]
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_weather_condition})
       
    
    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | college
Processing Record 2 of Set 1 | san andres
Processing Record 3 of Set 1 | iracoubo
Processing Record 4 of Set 1 | mount gambier
Processing Record 5 of Set 1 | harper
Processing Record 6 of Set 1 | barrow
Processing Record 7 of Set 1 | bluff
Processing Record 8 of Set 1 | iqaluit
Processing Record 9 of Set 1 | puerto narino
Processing Record 10 of Set 1 | vaini
Processing Record 11 of Set 1 | grindavik
Processing Record 12 of Set 1 | caravelas
Processing Record 13 of Set 1 | rikitea
Processing Record 14 of Set 1 | khatanga
Processing Record 15 of Set 1 | chokurdakh
Processing Record 16 of Set 1 | caxias
Processing Record 17 of Set 1 | tuktoyaktuk
Processing Record 18 of Set 1 | barreirinhas
Processing Record 19 of Set 1 | kodiak
Processing Record 20 of Set 1 | anlu
Processing Record 21 of Set 1 | airai
Processing Record 22 of Set 1 | nchelenge
Processing Record 23 of Set 1 | mataura
Processing Record 24 of Set 1 | tasiilaq
Processing Record 25 of Se

In [14]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,College,US,64.8569,-147.8028,56.80,74,75,3.44,broken clouds
1,San Andres,CO,12.5847,-81.7006,86.02,74,40,12.66,scattered clouds
2,Iracoubo,GF,5.4802,-53.2011,82.85,72,39,8.57,scattered clouds
3,Mount Gambier,AU,-37.8333,140.7667,47.23,85,62,8.05,broken clouds
4,Harper,LR,4.3750,-7.7169,77.50,66,66,8.68,broken clouds
...,...,...,...,...,...,...,...,...,...
712,Igrim,RU,63.1906,64.4162,62.65,51,80,6.67,broken clouds
713,Ust-Ilimsk,RU,58.0006,102.6619,57.74,73,61,1.74,broken clouds
714,Douentza,ML,14.9951,-2.9517,94.84,35,29,7.87,scattered clouds
715,Tubmanburg,LR,6.8708,-10.8172,79.56,69,95,6.62,overcast clouds


In [16]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")